In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('heart.csv')

In [6]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


#### For people who may not have access to their own medical data 

In [0]:
#age, sex, thalach using formulas to cal
# To estimate your maximum age-related heart rate, subtract your age from 220. For example, for a 50-year-old person, the estimated maximum age-related heart rate would be calculated as 220 – 50 years = 170 beats per minute (bpm).

In [6]:
#54, m

In [7]:
df_2 = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_2[(df_2['sex']==1)& (df_2['age'] ==54)].head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [43]:
df.chol.describe()
# 1 = male, 0 = female
df1 = df.groupby(by = ['age', 'sex'])['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'].mean()
df_2[(df_2['sex'] == 1) & (df_2['age'] == 54)].reset_index()

/tmp/ipykernel_1425/883282141.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = df.groupby(by = ['age', 'sex'])['cp', 'trestbps', 'chol', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'].mean()


,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,37,54,1,0.710526,127.184211,253.552632,0.0,0.368421,139.315789,0.394737,1.218421,1.263158,0.736842,2.631579,0.447368


In [35]:
df1

cp    trestbps        chol       fbs   restecg     exang  \
age sex                                                                   
29  1    1.000000  130.000000  204.000000  0.000000  0.000000  0.000000   
34  0    1.000000  118.000000  210.000000  0.000000  1.000000  0.000000   
    1    3.000000  118.000000  182.000000  0.000000  0.000000  0.000000   
35  0    0.000000  138.000000  183.000000  0.000000  1.000000  0.000000   
    1    0.363636  122.363636  218.727273  0.000000  0.727273  0.636364   
...           ...         ...         ...       ...       ...       ...   
70  1    0.642857  146.285714  251.857143  0.000000  0.500000  0.500000   
71  0    1.000000  124.363636  232.909091  0.363636  0.636364  0.000000   
74  0    1.000000  120.000000  269.000000  0.000000  0.000000  1.000000   
76  0    2.000000  140.000000  197.000000  0.000000  2.000000  0.000000   
77  1    0.000000  125.000000  304.000000  0.000000  0.000000  1.000000   

          oldpeak     slope        ca      thal    target  
age sex                                                    
29  1    0.000000  2.000000  0.000000  2.000000  1.000000  
34  0    0.700000  2.000000  0.000000  2.000000  1.000000  
    1    0.000000  2.000000  0.000000  2.000000  1.000000  
35  0    1.400000  2.000000  0.000000  2.000000  1.000000  
    1    0.581818  1.636364  0.000000  2.636364  0.363636  
...           ...       ...       ...       ...       ...  
70  1    2.050000  0.928571  1.071429  2.500000  0.214286  
71  0    0.690909  1.636364  0.909091  2.000000  1.000000  
74  0    0.200000  2.000000  1.000000  2.000000  1.000000  
76  0    1.100000  1.000000  0.000000  2.000000  1.000000  
77  1    0.000000  2.000000  3.000000  2.000000  0.000000  

[73 rows x 11 columns]

In [0]:
#function that takes in age and sex and returns a row of the mean data from that groupby df

In [0]:
# def mean_data:
#    user_age = input("Age?")
#    user_sex = input("Sex?")

In [11]:
df_mean_data = df.groupby(by = ['age', 'sex']).mean().reset_index()
df_mean_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,1,1.000000,130.000000,204.000000,0.000000,0.000000,202.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
1,34,0,1.000000,118.000000,210.000000,0.000000,1.000000,192.000000,0.000000,0.700000,2.000000,0.000000,2.000000,1.000000
2,34,1,3.000000,118.000000,182.000000,0.000000,0.000000,174.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000
3,35,0,0.000000,138.000000,183.000000,0.000000,1.000000,182.000000,0.000000,1.400000,2.000000,0.000000,2.000000,1.000000
4,35,1,0.363636,122.363636,218.727273,0.000000,0.727273,153.090909,0.636364,0.581818,1.636364,0.000000,2.636364,0.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,70,1,0.642857,146.285714,251.857143,0.000000,0.500000,121.500000,0.500000,2.050000,0.928571,1.071429,2.500000,0.214286
69,71,0,1.000000,124.363636,232.909091,0.363636,0.636364,136.909091,0.000000,0.690909,1.636364,0.909091,2.000000,1.000000
70,74,0,1.000000,120.000000,269.000000,0.000000,0.000000,121.000000,1.000000,0.200000,2.000000,1.000000,2.000000,1.000000
71,76,0,2.000000,140.000000,197.000000,0.000000,2.000000,116.000000,0.000000,1.100000,1.000000,0.000000,2.000000,1.000000


In [15]:
sex =""

if sex == "M" or sex == "Male":
    sex = 1
elif sex == "F" or sex == "Female":
    sex = 0

In [16]:
def age_sex(age, sex):
    df_mean_data[(df_mean_data['sex'] == sex) & (df_mean_data['age'] == age)]
    df_mean_data.head()

In [18]:
age_sex(54,1)